## ジオプロセシング ツールへのアクセス
ジオプロセシング ツールを検索して、コード内でアクセスする方法を以下に説明します。

* ジオプロセシング ツールの呼び出し方
* ツールとツールボックスの Python 表現
* ツールボックスのインポート
  - `Item` からツールボックスのインポート
  - ジオプロセシング サービスの URL からツールボックスのインポート
* ツールの署名、パラメーター、ドキュメント

## ジオプロセシング ツールの呼び出し方

 以下には ジオプロセシング ツールの実際の使い方を示していきます。
 ジオプロセシング ツールは、他のユーザーと共有できる Web ツールと言えます。ユーザーは、ツールをツールボックスで整理し、GIS 上で共有します。ジオプロセシング ツールは、他のアイテムを検索する場合と同様に検索できます。
ジオプロセシング ツールボックスを検索するには、アイテム の種類として `Geoprocessing Toolbox` を指定します。

In [4]:
import arcgis 
from arcgis.gis import GIS 
from IPython.display import display 

In [4]:
gis = GIS('https://www.arcgis.com', 'ユーザー名', 'パスワード')

In [5]:
toolboxes = gis.content.search('ocean','Geoprocessing Toolbox', 
                               outside_org=True, max_items=10)

In [6]:
for toolbox in toolboxes:
    display(toolbox)

<Item title:"ESRI_Currents_World/GPServer" type:Geoprocessing Toolbox owner:geodev1034admin>

<Item title:"CurrentTracerReport" type:Geoprocessing Toolbox owner:jingyi_huang_OpenOceansGlobal>

<Item title:"Hydrology" type:Geoprocessing Toolbox owner:Vblazek>

<Item title:"Farthest On Circle" type:Geoprocessing Toolbox owner:johnfry10>

<Item title:"SAR_User_Input" type:Geoprocessing Toolbox owner:ttmetoffice>

<Item title:"Ocean Currents" type:Geoprocessing Toolbox owner:esri_apac>

## ツールとツールボックスの Python 表現
ArcGIS API for Python では、ジオプロセシング ツールボックスは Python モジュールとして、個々のツールは Python 関数として表されます。Python 関数として表されるこれらのツールは、入力パラメーターのセットを取り込み、1 つ以上の出力値を返します。

カスタム ジオプロセシング ツールを使用するには、ユーザーはそのツールボックスをプログラム内のモジュールとしてインポートし、モジュール内で関数を呼び出すだけです。

## ツールボックスのインポート
`arcgis.geoprocessing` モジュールの関数である `import_toolbox()` によって、ジオプロセシング ツールボックスをネイティブ Python モジュールとしてインポートします。ツールボックスの場所として、GIS 内のジオプロセシング ツールボックス アイテム、またはジオプロセシング サービスの URL が挙げられます。

インポートしたモジュールで使用できる関数を呼び出して、これらのツールを呼び出すことができます。これら 2 つのソースからツールボックスをインポートする方法を見てみましょう。

### `Item` からツールボックスのインポート
　以下のコード スニペットは、上記の Ocean Currents ツールボックスをモジュールとしてインポートする方法を示しています。

In [6]:
from arcgis.geoprocessing import import_toolbox

In [8]:
ocean_currents_toolbox = toolboxes[5]
ocean_currents_toolbox

<Item title:"Ocean Currents" type:Geoprocessing Toolbox owner:esri_apac>

In [9]:
ocean_currents = import_toolbox(ocean_currents_toolbox)

`import_toolbox()` 関数は、ジオプロセシング ツールボックスを検査し、ツールボックス内の各ツールの関数を含む Python モジュールを動的に生成します。関数を呼び出すと、対応するジオプロセシング ツールが呼び出されます。

次のコード スニペットでは、Python の `inspect` モジュールを使用して、インポートされたモジュールのパブリック関数を一覧表示します。開発者は、IDE のインテリセンスを使用して、モジュール内の関数を検出します。

In [10]:
import inspect

# inspect を使用して、取得したツールボックスの関数を list 化
[ f[0] for f in inspect.getmembers(ocean_currents, inspect.isfunction) 
             if not f[0].startswith('_')]

['message_in_a_bottle']

In [11]:
help(ocean_currents.message_in_a_bottle)

Help on function message_in_a_bottle:

message_in_a_bottle(input_point: arcgis.features.feature.FeatureSet = {'geometryType': 'esriGeometryPoint', 'spatialReference': {'wkid': 4326}, 'Fields': [{'name': 'FID', 'type': 'esriFieldTypeOID', 'alias': 'FID'}, {'name': 'Shape', 'type': 'esriFieldTypeGeometry', 'alias': 'Shape'}, {'name': 'Id', 'type': 'esriFieldTypeInteger', 'alias': 'Id'}], 'fields': [{'name': 'FID', 'type': 'esriFieldTypeOID', 'alias': 'FID'}, {'name': 'Shape', 'type': 'esriFieldTypeGeometry', 'alias': 'Shape'}, {'name': 'Id', 'type': 'esriFieldTypeInteger', 'alias': 'Id'}]}, days: float = 150, gis=None, future=False) -> arcgis.features.feature.FeatureSet
            
    
    
    Parameters:
    
       input_point: Input Point (FeatureSet). Required parameter. 
    
       days: Days (float). Required parameter. 
    
            gis: Optional, the GIS on which this tool runs. If not specified, the active GIS is used.
    
    
            future: Optional, If True, a f

### ジオプロセシング サービスの URL からツールボックスのインポート
　以下の例では、ジオプロセシング サービスへのURLを使用して、ジオプロセシング ツールボックスをインポートしています。

In [12]:
toolbox_url = 'https://sampleserver1.arcgisonline.com/ArcGIS/rest/services/Elevation/ESRI_Elevation_World/GPServer/Viewshed'
elev = import_toolbox(toolbox_url)

## ツールの署名、パラメーター、ドキュメント
ジオプロセシング ツールを呼び出す関数には、そのツールに関するドキュメントが含まれています。このドキュメントは IDE のインテリセンスを使用して表示され、Python の help 関数を使用してアクセスすることもできます。

In [13]:
help(elev.viewshed) 

Help on function viewshed:

viewshed(input_observation_point: arcgis.features.feature.FeatureSet = {'geometryType': 'esriGeometryPoint', 'spatialReference': {'wkid': 54003}, 'Fields': [{'name': 'FID', 'type': 'esriFieldTypeOID', 'alias': 'FID'}, {'name': 'Shape', 'type': 'esriFieldTypeGeometry', 'alias': 'Shape'}, {'name': 'OffsetA', 'type': 'esriFieldTypeDouble', 'alias': 'OffsetA'}], 'fields': [{'name': 'FID', 'type': 'esriFieldTypeOID', 'alias': 'FID'}, {'name': 'Shape', 'type': 'esriFieldTypeGeometry', 'alias': 'Shape'}, {'name': 'OffsetA', 'type': 'esriFieldTypeDouble', 'alias': 'OffsetA'}]}, viewshed_distance: arcgis.geoprocessing._types.LinearUnit = {'distance': 15000, 'units': 'esriMeters'}, gis=None, future=False) -> arcgis.features.feature.FeatureSet
            
    
    
    Parameters:
    
       input_observation_point: Input Observation Point (FeatureSet). Required parameter. 
    
       viewshed_distance: Viewshed Distance (LinearUnit). Required parameter. 
    
     

上の例に示すように、ツール関数には、入力と出力を示すために、[タイプ ヒント](https://www.python.org/dev/peps/pep-0484/)を使用しています。関数シグネチャには入力パラメーターの既定値が含まれるため、呼び出し元は必要な場合を除き、値を指定する必要はありません。パラメーターのドキュメントには、各パラメーターの説明、期待されるタイプ、必須またはオプションのどちらであるかを含んでいます。パラメーターが入力値のリストを受け入れる場合、そのリストは Choise List としてドキュメントに含まれます。このドキュメントには、関数の戻り値のタイプと説明が含まれています。

次に、「ジオプロセシング ツールの使用(リンクは公開後)」に進み、Python スクリプトでこれらのツールを使用する方法を確認します。